In [11]:
import pandas as pd
import catboost
from verstack import NaNImputer
import numpy as np
import random
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
import keras_tuner as kt
import tensorflow_addons as tfa



In [3]:
data = pd.read_csv('flow_runs.csv', index_col=0)
data

,accuracy,algorithm,task,AutoCorrelation,CfsSubsetEval_DecisionStumpAUC,CfsSubsetEval_DecisionStumpErrRate,CfsSubsetEval_DecisionStumpKappa,CfsSubsetEval_NaiveBayesAUC,CfsSubsetEval_NaiveBayesErrRate,CfsSubsetEval_NaiveBayesKappa,...,RandomTreeDepth2AUC,RandomTreeDepth2ErrRate,RandomTreeDepth2Kappa,RandomTreeDepth3AUC,RandomTreeDepth3ErrRate,RandomTreeDepth3Kappa,StdvNominalAttDistinctValues,kNN1NAUC,kNN1NErrRate,kNN1NKappa
0,0.981401,Ensemble,9914,0.965482,0.933502,0.020045,0.804641,0.933502,0.020045,0.804641,...,0.867447,0.025212,0.748414,0.867447,0.025212,0.748414,0.0,0.654408,0.052077,0.403515
1,0.979748,Ensemble,9914,0.965482,0.933502,0.020045,0.804641,0.933502,0.020045,0.804641,...,0.867447,0.025212,0.748414,0.867447,0.025212,0.748414,0.0,0.654408,0.052077,0.403515
2,0.977268,Ensemble,9914,0.965482,0.933502,0.020045,0.804641,0.933502,0.020045,0.804641,...,0.867447,0.025212,0.748414,0.867447,0.025212,0.748414,0.0,0.654408,0.052077,0.403515
3,0.984088,Ensemble,9914,0.965482,0.933502,0.020045,0.804641,0.933502,0.020045,0.804641,...,0.867447,0.025212,0.748414,0.867447,0.025212,0.748414,0.0,0.654408,0.052077,0.403515
4,0.979955,Ensemble,9914,0.965482,0.933502,0.020045,0.804641,0.933502,0.020045,0.804641,...,0.867447,0.025212,0.748414,0.867447,0.025212,0.748414,0.0,0.654408,0.052077,0.403515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444216,0.833224,Ensemble,7293,0.720342,0.632301,0.162751,0.092566,0.632301,0.162751,0.092566,...,0.565331,0.247879,0.128400,0.565331,0.247879,0.128400,0.0,0.551255,0.250590,0.102600
444217,0.836338,Ensemble,7293,0.720342,0.632301,0.162751,0.092566,0.632301,0.162751,0.092566,...,0.565331,0.247879,0.128400,0.565331,0.247879,0.128400,0.0,0.551255,0.250590,0.102600
444218,0.834230,Ensemble,7293,0.720342,0.632301,0.162751,0.092566,0.632301,0.162751,0.092566,...,0.565331,0.247879,0.128400,0.565331,0.247879,0.128400,0.0,0.551255,0.250590,0.102600
444219,0.834366,Ensemble,7293,0.720342,0.632301,0.162751,0.092566,0.632301,0.162751,0.092566,...,0.565331,0.247879,0.128400,0.565331,0.247879,0.128400,0.0,0.551255,0.250590,0.102600


In [4]:
imputer = NaNImputer(conservative=True)
imputed_data = imputer.impute(data.groupby(['task', 'algorithm']).mean())
imputed_data

NaNImputer(conservative = True, n_feats = 10,            
           fix_string_nans = True, verbose = True,                
           multiprocessing_load = 3, fill_nans_in_pure_text = True,                    
           drop_empty_cols = True, drop_nan_cols_with_constant = True                        
           feature_selection = correlation)

Dataset dimensions:
 - rows:         7774
 - columns:      108
 - mb in memory: 6.44
 - NaN cols num: 88
--------------------------

Deploy multiprocessing with 12 parallel proceses



/opt/anaconda3/envs/tesi/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/envs/tesi/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/envs/tesi/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/opt/anaconda3/envs/tesi/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use panda

- CfsSubsetEval_DecisionStumpAUC: imputed 59 NaNs
- CfsSubsetEval_NaiveBayesErrRate: imputed 59 NaNs
- CfsSubsetEval_DecisionStumpErrRate: imputed 59 NaNs
- CfsSubsetEval_kNN1NErrRate:    imputed 59 NaNs
- CfsSubsetEval_kNN1NAUC:        imputed 59 NaNs
- CfsSubsetEval_NaiveBayesKappa: imputed 59 NaNs
- CfsSubsetEval_kNN1NKappa:      imputed 59 NaNs
- CfsSubsetEval_DecisionStumpKappa: imputed 59 NaNs
- CfsSubsetEval_NaiveBayesAUC:   imputed 59 NaNs
- DecisionStumpErrRate:          imputed 59 NaNs
- ClassEntropy:                  imputed 16 NaNs
- DecisionStumpAUC:              imputed 59 NaNs
- J48.00001.ErrRate:             imputed 59 NaNs
- J48.00001.AUC:                 imputed 59 NaNs
- J48.0001.AUC:                  imputed 59 NaNs
- J48.0001.ErrRate:              imputed 59 NaNs
- J48.001.AUC:                   imputed 59 NaNs
- J48.00001.Kappa:               imputed 59 NaNs
- J48.0001.Kappa:                imputed 59 NaNs
- J48.001.ErrRate:               imputed 59 NaNs
- Decisio

accuracy  AutoCorrelation  \
task   algorithm                                       
1      Ensemble            0.958165         0.606466   
       LinearRegression    0.671121         0.606466   
       LogisticRegression  0.988864         0.606466   
       NaiveBayes          0.974715         0.606466   
       NeuralNetwork       0.980407         0.606466   
...                             ...              ...   
167140 Ensemble            0.912264         0.375510   
       LogisticRegression  0.938046         0.375510   
       SVM                 0.808390         0.375510   
167141 Ensemble            0.925192         0.758352   
       SVM                 0.890160         0.758352   

                           CfsSubsetEval_DecisionStumpAUC  \
task   algorithm                                            
1      Ensemble                                  0.981204   
       LinearRegression                          0.981204   
       LogisticRegression                        0.981204   
       NaiveBayes                                0.981204   
       NeuralNetwork                             0.981204   
...                                                   ...   
167140 Ensemble                                  1.000035   
       LogisticRegression                        1.000035   
       SVM                                       1.000035   
167141 Ensemble                                  1.000035   
       SVM                                       1.000035   

                           CfsSubsetEval_DecisionStumpErrRate  \
task   algorithm                                                
1      Ensemble                                      0.023385   
       LinearRegression                              0.023385   
       LogisticRegression                            0.023385   
       NaiveBayes                                    0.023385   
       NeuralNetwork                                 0.023385   
...                                                       ...   
167140 Ensemble                                      0.758119   
       LogisticRegression                            0.758119   
       SVM                                           0.758119   
167141 Ensemble                                      0.758119   
       SVM                                           0.758119   

                           CfsSubsetEval_DecisionStumpKappa  \
task   algorithm                                              
1      Ensemble                                    0.941800   
       LinearRegression                            0.941800   
       LogisticRegression                          0.941800   
       NaiveBayes                                  0.941800   
       NeuralNetwork                               0.941800   
...                                                     ...   
167140 Ensemble                                    0.999974   
       LogisticRegression                          0.999974   
       SVM                                         0.999974   
167141 Ensemble                                    0.999974   
       SVM                                         0.999974   

                           CfsSubsetEval_NaiveBayesAUC  \
task   algorithm                                         
1      Ensemble                               0.981204   
       LinearRegression                       0.981204   
       LogisticRegression                     0.981204   
       NaiveBayes                             0.981204   
       NeuralNetwork                          0.981204   
...                                                ...   
167140 Ensemble                               1.000048   
       LogisticRegression                     1.000048   
       SVM                                    1.000048   
167141 Ensemble                               1.000048   
       SVM                                    1.000048   

                           CfsSubsetEval_NaiveBayesErrRate  \
task   algorithm     

In [5]:
def test_train_split(dataset, train_size_tasks=0.8, one_hot=False):
    X = dataset.drop(['accuracy'], axis=1)
    y = dataset['accuracy']
    task_list = dataset.index.levels[0].to_list()
    train_tasks = random.sample(task_list, int(train_size_tasks*len(task_list)))
    test_tasks = list(set(task_list)-set(train_tasks))
    X_train = X.loc[train_tasks].reset_index().drop(['task'], axis=1)
    y_train = y.loc[train_tasks].reset_index().drop(['task', 'algorithm'], axis=1)
    X_test = X.loc[test_tasks].reset_index().drop(['task'], axis=1)
    y_test = y.loc[test_tasks].reset_index().drop(['task', 'algorithm'], axis=1)
    if one_hot:
        X_train = pd.concat([X_train.drop(['algorithm'], axis=1), pd.get_dummies(X_train['algorithm'])], axis=1)
        X_test = pd.concat([X_test.drop(['algorithm'], axis=1), pd.get_dummies(X_test['algorithm'])], axis=1)
    return X_train, y_train, X_test, y_test

In [6]:
no_algo = data.groupby(['task']).mean().reset_index()
X_noalgo = no_algo.drop(['accuracy', 'task'], axis=1)
y_noalgo = no_algo['accuracy']

In [7]:
X_train, y_train, X_test, y_test = test_train_split(imputed_data)


In [15]:
X_train

,AutoCorrelation,CfsSubsetEval_DecisionStumpAUC,CfsSubsetEval_DecisionStumpErrRate,CfsSubsetEval_DecisionStumpKappa,CfsSubsetEval_NaiveBayesAUC,CfsSubsetEval_NaiveBayesErrRate,CfsSubsetEval_NaiveBayesKappa,CfsSubsetEval_kNN1NAUC,CfsSubsetEval_kNN1NErrRate,CfsSubsetEval_kNN1NKappa,...,kNN1NErrRate,kNN1NKappa,DecisionTree,Ensemble,LinearRegression,LogisticRegression,NaiveBayes,NeuralNetwork,SVM,kNearestNeighbor
0,0.466079,-1.892390,0.272158,-1.708239,-1.930515,0.284572,-1.724752,-1.899887,0.282365,-1.717544,...,0.509358,-1.208760,8.408191,-0.439182,-0.401504,-0.394395,-0.422779,-0.419309,-0.279572,-0.464171
1,0.466079,-1.892390,0.272158,-1.708239,-1.930515,0.284572,-1.724752,-1.899887,0.282365,-1.717544,...,0.509358,-1.208760,-0.118932,2.276961,-0.401504,-0.394395,-0.422779,-0.419309,-0.279572,-0.464171
2,0.466079,-1.892390,0.272158,-1.708239,-1.930515,0.284572,-1.724752,-1.899887,0.282365,-1.717544,...,0.509358,-1.208760,-0.118932,-0.439182,2.490637,-0.394395,-0.422779,-0.419309,-0.279572,-0.464171
3,0.466079,-1.892390,0.272158,-1.708239,-1.930515,0.284572,-1.724752,-1.899887,0.282365,-1.717544,...,0.509358,-1.208760,-0.118932,-0.439182,-0.401504,2.535531,-0.422779,-0.419309,-0.279572,-0.464171
4,0.466079,-1.892390,0.272158,-1.708239,-1.930515,0.284572,-1.724752,-1.899887,0.282365,-1.717544,...,0.509358,-1.208760,-0.118932,-0.439182,-0.401504,-0.394395,2.365302,-0.419309,-0.279572,-0.464171
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6161,-0.248693,1.370125,-1.383249,1.561906,1.354976,-1.379468,1.554050,1.355605,-1.374907,1.549620,...,-1.329575,1.583097,8.408191,-0.439182,-0.401504,-0.394395,-0.422779,-0.419309,-0.279572,-0.464171
6162,-0.248693,1.370125,-1.383249,1.561906,1.354976,-1.379468,1.554050,1.355605,-1.374907,1.549620,...,-1.329575,1.583097,-0.118932,2.276961,-0.401504,-0.394395,-0.422779,-0.419309,-0.279572,-0.464171
6163,-0.248693,1.370125,-1.383249,1.561906,1.354976,-1.379468,1.554050,1.355605,-1.374907,1.549620,...,-1.329575,1.583097,-0.118932,-0.439182,-0.401504,-0.394395,-0.422779,2.384876,-0.279572,-0.464171
6164,-0.248693,1.370125,-1.383249,1.561906,1.354976,-1.379468,1.554050,1.355605,-1.374907,1.549620,...,-1.329575,1.583097,-0.118932,-0.439182,-0.401504,-0.394395,-0.422779,-0.419309,3.576896,-0.464171


In [9]:
model = catboost.CatBoostRegressor(cat_features=['algorithm'])
model.fit(X_train, y_train)

Learning rate set to 0.054719
0:	learn: 0.1595820	total: 62ms	remaining: 1m 1s
1:	learn: 0.1547650	total: 71.9ms	remaining: 35.9s
2:	learn: 0.1504619	total: 81.3ms	remaining: 27s
3:	learn: 0.1462161	total: 89.3ms	remaining: 22.2s
4:	learn: 0.1421959	total: 97.5ms	remaining: 19.4s
5:	learn: 0.1386340	total: 106ms	remaining: 17.5s
6:	learn: 0.1353411	total: 114ms	remaining: 16.2s
7:	learn: 0.1322284	total: 122ms	remaining: 15.2s
8:	learn: 0.1292208	total: 130ms	remaining: 14.3s
9:	learn: 0.1262932	total: 140ms	remaining: 13.8s
10:	learn: 0.1237797	total: 150ms	remaining: 13.5s
11:	learn: 0.1212403	total: 157ms	remaining: 12.9s
12:	learn: 0.1186425	total: 170ms	remaining: 12.9s
13:	learn: 0.1164712	total: 179ms	remaining: 12.6s
14:	learn: 0.1145274	total: 187ms	remaining: 12.3s
15:	learn: 0.1124811	total: 196ms	remaining: 12s
16:	learn: 0.1106623	total: 204ms	remaining: 11.8s
17:	learn: 0.1088668	total: 214ms	remaining: 11.7s
18:	learn: 0.1073540	total: 222ms	remaining: 11.5s
19:	learn: 0

In [12]:
X_train, y_train, X_test, y_test = test_train_split(imputed_data, one_hot=True)
normalizer = tf.keras.layers.Normalization(axis=-1, name='normalizer')
normalizer.adapt(X_train)

2022-04-07 11:32:36.159775: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
from sklearn.preprocessing import StandardScaler

X_train[X_train.columns] = StandardScaler().fit_transform(
    X_train[X_train.columns])
X_test[X_test.columns] = StandardScaler().fit_transform(X_test[X_test.columns])

X_train.describe().loc["max"].max()

70.83932778091128

In [14]:

def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(X_train.shape[1],)))
    model.add(normalizer)
    for n_layer in range(hp.Int('n_layers', 1, 6, default=2)):
        model.add(
            tf.keras.layers.Dense(hp.Int('n_units_' + str(n_layer), 1, 512),
                               activation=hp.Choice(
                                   'activation', ['relu', 'sigmoid', 'tanh'])))
        model.add(
            tf.keras.layers.Dropout(
                hp.Float('dropout_' + str(n_layer), 0, 0.5, step=0.05)))
    model.add(tf.keras.layers.Dense(1))
    model.compile(loss='mse',
                  optimizer='adam',
                  metrics=[tfa.metrics.RSquare(dtype=tf.float32, y_shape=(1,))])
    return model


tuner = kt.Hyperband(build_model,
                     objective=kt.Objective("r_square", "max"),
                     max_epochs=100,
                     directory='/tmp/keras_tuner',
                     project_name='lastduel',
                     overwrite=True)
tuner.search(X_train,
             y_train,
             epochs=10,
             validation_split=0.2,
             callbacks=[tf.keras.callbacks.TensorBoard("/tmp/tb_logs")])


Trial 6 Complete [00h 00m 01s]
r_square: nan

Best r_square So Far: nan
Total elapsed time: 00h 00m 11s

Search: Running Trial #7

Hyperparameter    |Value             |Best Value So Far 
n_layers          |5                 |3                 
n_units_0         |372               |80                
activation        |sigmoid           |relu              
dropout_0         |0.1               |0.2               
n_units_1         |375               |462               
dropout_1         |0.05              |0.3               
n_units_2         |275               |1                 
dropout_2         |0.2               |0                 
n_units_3         |399               |None              
dropout_3         |0.05              |None              
n_units_4         |503               |None              
dropout_4         |0.05              |None              
tuner/epochs      |2                 |2                 
tuner/initial_e...|0                 |0                 
tuner/bracket 

KeyboardInterrupt: 

In [ ]:
tuner.get_best_models()[0].summary()

NotFoundError: Unsuccessful TensorSliceReader constructor: Failed to find any matching files for /tmp/keras_tuner/lastduel/trial_ce36621bde5d718ad8ce0f61adcd71cf/checkpoints/epoch_None/checkpoint